In [1]:
using Pkg
Pkg.activate(".")
Pkg.add("Plots")
Pkg.add("BenchmarkTools")

  Activating project at `~`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`


### KR1: 

Shown or demonstrated the hierarchy of Julia's type hierarchy using the command subtypes(). Start from Number and use subtypes() to explore from abstract types down to specific types. Use supertype() to determine the parent abstract type.

In [2]:
subtypes(Number)

2-element Vector{Any}:
 Complex
 Real

In [3]:
subtypes(Complex)

Type[]

In [4]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

In [5]:
subtypes(AbstractFloat)

4-element Vector{Any}:
 BigFloat
 Float16
 Float32
 Float64

In [6]:
subtypes(Float64)

Type[]

In [7]:
subtypes(AbstractIrrational)

1-element Vector{Any}:
 Irrational

In [8]:
supertype(Irrational)

AbstractIrrational

### KR2:

Implemented and used at least one own composite type via struct. Generate two more versions that are mutable type and type-parametrized of the custom-built type.

#### structs

In [61]:
struct Person
    myname::String
    age::Int64
end

function printMyActivity(self::Person)
    println("I am $(self.myname), $(self.age) years old")
end

Ryan = Person("Ryan Vitalez", 24)

printMyActivity(Ryan)

I am Ryan Vitalez, 24 years old


In [63]:
println("typeof(Ryan) = $(typeof(Ryan)).");
println("Ryan = $(Ryan)");

typeof(Ryan) = Person.
Ryan = Person("Ryan Vitalez", 24)


In [64]:
Person() = Person("Ryan Vitalez", 24)

Person

In [67]:
P = Person()

Person("Ryan Vitalez", 24)

In [68]:
P.myname

"Ryan Vitalez"

In [69]:
P.age

24

#### Creating mutable structs

In [71]:
mutable struct MPerson
    myname::String
    age::Int64
end

MPerson() = MPerson("Ryan Vitalez", 24)

MPerson

In [73]:
MP = MPerson("Ryan Vitalez", 24)

MPerson("Ryan Vitalez", 24)

In [74]:
MP.myname

"Ryan Vitalez"

In [75]:
MP.age

24

In [76]:
typeof(MP)

MPerson

In [77]:
MP = ("Ryan M. Vitalez", 24)

("Ryan M. Vitalez", 24)

### KR3

Demonstrated type inference in Julia. Generator expressions may be used for this.


In [85]:
[x^3 for x in 1:5]

5-element Vector{Int64}:
   1
   8
  27
  64
 125

In [86]:
[x^2 for x in 1.0:5.0]

5-element Vector{Float64}:
  1.0
  4.0
  9.0
 16.0
 25.0

### KR4

Created a function with inherent type-instability. Create a version of the function with fixed type-instability issues.

### KR 5

Demonstration of how @code_warntype can be useful in detecting type-instability.


In [91]:
γ(t) = t > 0 ? 0 : t

γ (generic function with 1 method)

In [92]:
γ(-2)

-2

In [93]:
γ(2)

0

In [94]:
γ(-2.0)

-2.0

In [95]:
γ(2.0)

0

In [96]:
println("typeof(γ(-2)) = $(typeof(γ(-2))).")
println("typeof(γ(2)) = $(typeof(γ(2))).")

println("typeof(γ(-2.0)) = $(typeof(γ(-2.0))).")
println("typeof(γ(2.0)) = $(typeof(γ(2.0))).")

typeof(γ(-2)) = Int64.
typeof(γ(2)) = Int64.
typeof(γ(-2.0)) = Float64.
typeof(γ(2.0)) = Int64.


In [108]:
@code_warntype γ(-2)

MethodInstance for γ(::Int64)
  from γ(t) in Main at In[91]:1
Arguments
  #self#::Core.Const(γ)
  t::Int64
Body::Int64
1 ─ %1 = (t > 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return t



In [109]:
@code_warntype γ(-2.0)

MethodInstance for γ(::Float64)
  from γ(t) in Main at In[91]:1
Arguments
  #self#::Core.Const(γ)
  t::Float64
Body::Union{Float64, Int64}
1 ─ %1 = (t > 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return t



In [97]:
t = 2.0
n = 2

2

In [110]:
t

2.0

In [111]:
n

2

In [112]:
γ₀(t) = t > 0 ? zero(t) : t

γ₀ (generic function with 1 method)

In [113]:
γ₀(-2)

-2

In [114]:
γ₀(2)

0

In [115]:
@code_warntype γ₀(-2.0)

MethodInstance for γ₀(::Float64)
  from γ₀(t) in Main at In[112]:1
Arguments
  #self#::Core.Const(γ₀)
  t::Float64
Body::Float64
1 ─ %1 = (t > 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero(t)::Core.Const(0.0)
└──      return %3
3 ─      return t



### KR 6 

Demonstration of how Arrays containing ambiguous/abstract types often results to slow execution of codes. The BenchmarkTools may be useful in this part.


In [121]:
## I can't think of another function to use so I used the one from the discussion. 

function sumsqrtn_naive(n)
    ret = 0
    for x in 1:n
        ret = ret + sqrt(x)
    end
end

sumsqrtn_naive (generic function with 1 method)

In [132]:
function sumsqrtn_clean(n)
    ret = 0.0
    for x in 1:n
        ret = ret + sqrt(x)
    end
end

sumsqrtn_clean (generic function with 1 method)

In [136]:
using BenchmarkTools
mark1 = @benchmark sumsqrtn_naive(10_000_000)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.125 ns … 10.375 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.208 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.203 ns ±  0.108 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                ▇             █▇             ▃             ▂ ▂
  █▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█ █
  2.12 ns      Histogram: log(frequency) by time     2.29 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [137]:
mark2 = @benchmark sumsqrtn_clean(10_000_000)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.083 ns … 7.458 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.208 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.201 ns ± 0.074 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                              ▁             █▁               
  ▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  2.08 ns        Histogram: frequency by time       2.25 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [138]:
mean(mark1.times)/mean(mark2.times) ## I used mean because their result will be one if I use median

1.0010258865851185

In [139]:
@code_warntype sumsqrtn_naive(10)

MethodInstance for sumsqrtn_naive(::Int64)
  from sumsqrtn_naive(n) in Main at In[121]:1
Arguments
  #self#::Core.Const(sumsqrtn_naive)
  n::Int64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  ret::Union{Float64, Int64}
  x::Int64
Body::Nothing
1 ─       (ret = 0)
│   %2  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = ret::Union{Float64, Int64}
│   %11 = Main.sqrt(x)::Float64
│         (ret = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return nothing



In [140]:
@code_warntype sumsqrtn_clean(10)

MethodInstance for sumsqrtn_clean(::Int64)
  from sumsqrtn_clean(n) in Main at In[132]:1
Arguments
  #self#::Core.Const(sumsqrtn_clean)
  n::Int64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  ret::Float64
  x::Int64
Body::Nothing
1 ─       (ret = 0.0)
│   %2  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = ret::Float64
│   %11 = Main.sqrt(x)::Float64
│         (ret = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return nothing



In [142]:
function sumsqrtn_cleanr(n)
    ret = 0.0
    for x in 1:n
        ret = ret + sqrt(1.0*n)
    end
end

sumsqrtn_cleanr (generic function with 1 method)

In [143]:
mark3 = @benchmark sumsqrtn_cleanr(10_000_000)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.083 ns … 33.250 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.208 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.211 ns ±  0.372 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                     ▁        █▁                              
  ▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▂ ▂
  2.08 ns        Histogram: frequency by time        2.33 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [145]:
@code_warntype sumsqrtn_cleanr(10)

MethodInstance for sumsqrtn_cleanr(::Int64)
  from sumsqrtn_cleanr(n) in Main at In[142]:1
Arguments
  #self#::Core.Const(sumsqrtn_cleanr)
  n::Int64
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  ret::Float64
  x::Int64
Body::Nothing
1 ─       (ret = 0.0)
│   %2  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = ret::Float64
│   %11 = (1.0 * n)::Float64
│   %12 = Main.sqrt(%11)::Float64
│         (ret = %10 + %12)
│         (@_3 = Base.iterate(%2, %9))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return nothing



In [146]:
println("mark1/mark2 = $(mean(mark1.times) / mean(mark2.times))")
println("mark2/mark3 = $(mean(mark2.times) / mean(mark3.times))")

mark1/mark2 = 1.0010258865851185
mark2/mark3 = 0.9952774387169163
